In [1]:
from json import loads
import numpy as np
import sympy
from tools import toCopyString, fromCopyString

In [4]:
# matrix([1,2,3],[0,1,2],[0,0,4])
# matrix([-2,1],[2,-1],[0,1])

# loads('matrix([1,2,3],[0,1,2],[0,0,4])'.replace('matrix(', '[').replace(')', ']'))
fromCopyString('matrix([1,2,3],[0,1,2],[0,0,4])')

[[1, 2, 3], [0, 1, 2], [0, 0, 4]]

In [7]:
A = sympy.Matrix(fromCopyString('matrix([1,2,3],[0,1,2],[0,0,4])'))
B = sympy.Matrix(fromCopyString('matrix([-2,1],[2,-1],[0,1])'))
L = sympy.Matrix(fromCopyString('matrix([-1],[-2],[-1])'))
r = sympy.Matrix(fromCopyString('matrix([1],[2])'))
A, B, L, r

(Matrix([
 [1, 2, 3],
 [0, 1, 2],
 [0, 0, 4]]),
 Matrix([
 [-2,  1],
 [ 2, -1],
 [ 0,  1]]),
 Matrix([
 [-1],
 [-2],
 [-1]]),
 Matrix([
 [1],
 [2]]))

0: Матрица управляемости

In [21]:
Scontrol = sympy.zeros(3,6)
Scontrol[:, 0:2] = B 
Scontrol[:, 2:4] = A*B
Scontrol[:, 4:] = A**2*B
toCopyString(Scontrol)

'matrix([-2,1,2,2,6,16],[2,-1,2,1,2,9],[0,1,0,4,0,16])'

1: Управляемость системы

In [22]:
# slice out to (3,3)
ScontrolSquared = Scontrol[:, 0:3]
detScontrol = ScontrolSquared.det()
detScontrol, 'да' if detScontrol != 0 else 'нет'

(8, 'да')

2: Коэффициенты характеристического полинома разомкнутой системы (в порядке возрастания степеней от свободного члена)

In [23]:
l = sympy.symbols('lambda')
I = sympy.eye(3)
detChPolyOpen = (l*I - A).det()
c0 = detChPolyOpen.coeff(l, 0)
c1 = detChPolyOpen.coeff(l, 1)
c2 = detChPolyOpen.coeff(l, 2)
ChPolyOpen = sympy.Matrix([
    c0,
    c1,
    c2
    ])
toCopyString(ChPolyOpen)

'matrix([-4],[9],[-6])'

3: Коэффициенты характеристического полинома замкнутой системы (в порядке возрастания степеней от свободного члена)

In [24]:
ChPolyClosed = sympy.Matrix([
    [-(L[0]*L[1]*L[2])],
    [L[0]*L[1] + L[0]*L[2] + L[1]*L[2]],
    [-(L[0] + L[1] + L[2])]
    ])
toCopyString(ChPolyClosed)

'matrix([2],[5],[4])'

4: Матрица D^-1

In [26]:
b = B*r
d1 = A**2*b+c2*A*b+c1*b
d2 = A*b+c2*b
d3 = b
D = sympy.eye(3)
D[:, 0] = d1
D[:, 1] = d2
D[:, 2] = d3
Dinv = D.inv().evalf()
# evaluate Dinv
toCopyString(Dinv)

'matrix([0.125000000000000,-0.187500000000000,0],[0.125000000000000,0.0625000000000000,0],[0.125000000000000,0.312500000000000,0.500000000000000])'

5: Скалярное управление v

In [27]:
GpT = (ChPolyOpen - ChPolyClosed).T
v = GpT*Dinv
toCopyString(v)

'matrix([-1.50000000000000,-1.75000000000000,-5.00000000000000])'

6: Векторное управление u

In [29]:
u = r*v
toCopyString(u)

'matrix([-1.50000000000000,-1.75000000000000,-5.00000000000000],[-3.00000000000000,-3.50000000000000,-10.0000000000000])'